In [ ]:
  #install essential packages
! git clone https://github.com/xyyzh/NLP_final_project.git
!pip install transformers

fatal: destination path 'NLP_final_project' already exists and is not an empty directory.


In [ ]:
#import
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import torch
import gc
import torch.nn as nn
from sklearn.metrics import mean_absolute_error, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
%matplotlib inline

import pandas
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#load the GPU
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [ ]:
#Training metadata
epochs = 10
batch_size=16

In [ ]:
def convert_train_data_of(lang):
  if lang == 'en':
    path = "NLP_final_project/train/dataset_en_train_trimmed.json"
  elif lang == 'fr':
    path = "NLP_final_project/train/dataset_fr_train_trimmed.json"
  elif lang == 'de':
    path = "NLP_final_project/train/dataset_de_train_trimmed.json"
  elif lang == 'es':
    path = "NLP_final_project/train/dataset_es_train_trimmed.json"
  elif lang == 'zh':
    path = "NLP_final_project/train/dataset_zh_train_trimmed.json"
  elif lang == 'ja':
    path = "NLP_final_project/train/dataset_ja_train_trimmed.json"
  else:
    print("Not a valid option")
    return -1

  training = pd.read_json(path, lines=True)
  training.drop(["product_id", "reviewer_id", "language", "product_category", "review_title"], inplace=True, axis=1)
  Review = training.review_body.values

  Star = [int(string) - 1 for string in training.stars.values]
  train_text, val_text, train_labels, val_labels = train_test_split(Review, Star, random_state=2021, test_size=0.3, stratify=Star)
  mlength = 100
  # tokenize and encode sequences in the training set
  tokens_train = tokenizer.batch_encode_plus(
      train_text.tolist(),
      max_length = mlength,
      padding=True,
      truncation=True
  )

# tokenize and encode sequences in the validation set
  tokens_val = tokenizer.batch_encode_plus(
      val_text.tolist(),
      max_length = mlength,
      padding=True,
      truncation=True
  )  

  train_seq = torch.tensor(tokens_train['input_ids'])
  train_mask = torch.tensor(tokens_train['attention_mask'])
  train_y = torch.tensor(train_labels)

  val_seq = torch.tensor(tokens_val['input_ids'])
  val_mask = torch.tensor(tokens_val['attention_mask'])
  val_y = torch.tensor(val_labels)

  # wrap tensors
  train_data = TensorDataset(train_seq, train_mask, train_y)

  # sampler for sampling the data during training
  train_sampler = RandomSampler(train_data)

  # dataLoader for train set
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  # wrap tensors
  val_data = TensorDataset(val_seq, val_mask, val_y)

  # sampler for sampling the data during training
  val_sampler = SequentialSampler(val_data)

  # dataLoader for validation set
  val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

  return train_dataloader, val_dataloader


In [ ]:
def convert_test_data_of(lang):
  if lang == 'en':
    path = "NLP_final_project/test/dataset_en_test.json"
  elif lang == 'fr':
    path = "NLP_final_project/test/dataset_fr_test.json"
  elif lang == 'de':
    path = "NLP_final_project/test/dataset_de_test.json"
  elif lang == 'es':
    path = "NLP_final_project/test/dataset_es_test.json"
  elif lang == 'zh':
    path = "NLP_final_project/test/dataset_zh_test.json"
  elif lang == 'ja':
    path = "NLP_final_project/test/dataset_ja_test.json"
  else:
    print("Not a valid option")
    return -1
  test = pd.read_json(path, lines=True)
  test.drop(["product_id", "reviewer_id", "language", "product_category", "review_title"], inplace=True, axis=1)
  test_text=test.review_body.values
  test_labels=[int(string) - 1 for string in test.stars.values]
  mlength=100
  tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = mlength,
    padding=True,
    truncation=True
  )

  test_seq = torch.tensor(tokens_test['input_ids'])
  test_mask = torch.tensor(tokens_test['attention_mask'])
  test_y = torch.tensor(test_labels)

  # wrap tensors
  test_data = TensorDataset(test_seq, test_mask, test_y)

  # sampler for sampling the data during training
  test_sampler = SequentialSampler(test_data)

  # dataLoader for validation set
  test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=batch_size)

  return test_dataloader, test_y

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)
train_dataloader, val_dataloader = convert_train_data_of('fr')

In [ ]:
class BERT(nn.Module):
    
    def __init__(self, bert):
      
      super(BERT, self).__init__()

      self.bert = bert 
      
      self.classifier = nn.Sequential(
          nn.Linear(768, 512),
          nn.ReLU(),
          nn.Linear(512, 5)
      )

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      outputs = self.bert(input_ids = sent_id, attention_mask = mask)
      
     # Extract the last hidden state of the token `[CLS]` for classification task
      last_hidden_state_cls = outputs[0][:, 0, :]

      # Feed input to classifier to compute logits
      logits = self.classifier(last_hidden_state_cls)

      return logits

In [ ]:
def load_pretrained_model():

  bert=BertModel.from_pretrained('bert-base-multilingual-cased', return_dict=False)
  model = BERT(bert)
  '''
  for param in bert.parameters():
    param.requires_grad = False
    # pass the pre-trained BERT to our define architecture
  model = BERT_Arch(bert)
  '''
  # push the model to GPU

  model = model.to(device)  
  return model

In [ ]:
#load tokenizer models


model = load_pretrained_model()

optimizer = AdamW(model.parameters(), lr = 2e-5)  

cross_entropy  = nn.CrossEntropyLoss() 


scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, # Default value
                                            num_training_steps=len(train_dataloader)*epochs)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# function to train the model
def train(train_dataloader):
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  total, correct=0, 0
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    total += labels.size(0)
    _, predicted = torch.max(preds.data, 1)
    correct += (predicted == labels).sum().item()

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()
    scheduler.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  acc = 100*correct/total
  #returns the loss and predictions
  return avg_loss, total_preds, acc

In [ ]:
# function for evaluating the model
def evaluate(val_dataloader):
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []
  total, correct=0, 0


  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total += labels.size(0)
      _, predicted = torch.max(preds.data, 1)
      correct += (predicted == labels).sum().item()
      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 
  preds = np.argmax(preds, axis = 1)
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  acc = 100*correct/total

  return avg_loss, total_preds, acc

In [ ]:
def train_model_on(train_dataloader, val_dataloader, epochs):
  best_valid_loss = float('inf')

  # empty lists to store training and validation loss of each epoch
  train_losses=[]
  valid_losses=[]

  #for each epoch
  for epoch in range(epochs):
     
      print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
      #train model
      train_loss, _, acc1 = train(train_dataloader)

      #evaluate model
      valid_loss, preds, acc2 = evaluate(val_dataloader)
    

      #save the best model
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)
      print(f'\nTraining Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')
      print(f'Training Accuracy: {acc1:.3f}')
      print(f'Validation Accuracy: {acc2:.3f}')

In [ ]:
def test_model_on(dataloader, test_y):
  print("\nTesting...")
  path = 'saved_weights.pt'
  model.load_state_dict(torch.load(path))
  print("Model state loaded from file "+path)
  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []
  total, correct=0, 0

  # iterate over batches
  for step,batch in enumerate(dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0 :
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)
      total += labels.size(0)
      _, predicted = torch.max(preds.data, 1)
      correct += (predicted == labels).sum().item()
      predicted = predicted.detach().cpu().numpy()
      total_preds.append(predicted)
      
  
  total_preds  = np.concatenate(total_preds, axis=0)
  test_y = [i+1 for i in test_y]
  total_preds = [i+1 for i in total_preds]
  acc = 100*correct/total
  MAE = mean_absolute_error(test_y, total_preds)
  report = classification_report(test_y, total_preds)
  return acc, MAE, report

In [ ]:
train_model_on(train_dataloader, val_dataloader, epochs)


 Epoch 1 / 10
  Batch    50  of  1,094.
  Batch   100  of  1,094.
  Batch   150  of  1,094.
  Batch   200  of  1,094.
  Batch   250  of  1,094.
  Batch   300  of  1,094.
  Batch   350  of  1,094.
  Batch   400  of  1,094.
  Batch   450  of  1,094.
  Batch   500  of  1,094.
  Batch   550  of  1,094.
  Batch   600  of  1,094.
  Batch   650  of  1,094.
  Batch   700  of  1,094.
  Batch   750  of  1,094.
  Batch   800  of  1,094.
  Batch   850  of  1,094.
  Batch   900  of  1,094.
  Batch   950  of  1,094.
  Batch 1,000  of  1,094.
  Batch 1,050  of  1,094.

Evaluating...
  Batch    50  of    469.
  Batch   100  of    469.
  Batch   150  of    469.
  Batch   200  of    469.
  Batch   250  of    469.
  Batch   300  of    469.
  Batch   350  of    469.
  Batch   400  of    469.
  Batch   450  of    469.

Training Loss: 1.256
Validation Loss: 1.121
Training Accuracy: 43.114
Validation Accuracy: 50.053

 Epoch 2 / 10
  Batch    50  of  1,094.
  Batch   100  of  1,094.
  Batch   150  of  1,094

In [ ]:
print("Start running the model on English")
test_dataloader, test_y=convert_test_data_of('en')
acc3, mae, report=test_model_on(test_dataloader, test_y)
print(f'Test Accuracy: {acc3:.3f}')
print(f'MAE: {mae:.3f}')
print(report)

Start running the model on English

Testing...
Model state loaded from file saved_weights.pt
  Batch    50  of    313.
  Batch   100  of    313.
  Batch   150  of    313.
  Batch   200  of    313.
  Batch   250  of    313.
  Batch   300  of    313.
Test Accuracy: 47.620
MAE: 0.703
              precision    recall  f1-score   support

           1       0.55      0.63      0.59      1000
           2       0.39      0.36      0.38      1000
           3       0.35      0.44      0.39      1000
           4       0.45      0.32      0.37      1000
           5       0.66      0.62      0.64      1000

    accuracy                           0.48      5000
   macro avg       0.48      0.48      0.47      5000
weighted avg       0.48      0.48      0.47      5000



In [ ]:
print("Start running the model on French")
test_dataloader, test_y=convert_test_data_of('fr')
acc, mae, report=test_model_on(test_dataloader, test_y)
print(f'Test Accuracy: {acc:.3f}')
print(f'MAE: {mae:.3f}')
print(report)
# print(test_dataloader)

Start running the model on French

Testing...
Model state loaded from file saved_weights.pt
  Batch    50  of    313.
  Batch   100  of    313.
  Batch   150  of    313.
  Batch   200  of    313.
  Batch   250  of    313.
  Batch   300  of    313.
Test Accuracy: 52.360
MAE: 0.589
              precision    recall  f1-score   support

           1       0.60      0.78      0.67      1000
           2       0.48      0.34      0.40      1000
           3       0.43      0.43      0.43      1000
           4       0.45      0.47      0.46      1000
           5       0.64      0.59      0.62      1000

    accuracy                           0.52      5000
   macro avg       0.52      0.52      0.52      5000
weighted avg       0.52      0.52      0.52      5000



In [ ]:
print("Start running the model on Germany")
test_dataloader, test_y=convert_test_data_of('de')
acc, mae, report=test_model_on(test_dataloader, test_y)
print(f'Test Accuracy: {acc:.3f}')
print(f'MAE: {mae:.3f}')
print(report)

Start running the model on Germany

Testing...
Model state loaded from file saved_weights.pt
  Batch    50  of    313.
  Batch   100  of    313.
  Batch   150  of    313.
  Batch   200  of    313.
  Batch   250  of    313.
  Batch   300  of    313.
Test Accuracy: 43.260
MAE: 0.781
              precision    recall  f1-score   support

           1       0.59      0.45      0.51      1000
           2       0.36      0.36      0.36      1000
           3       0.33      0.55      0.41      1000
           4       0.39      0.32      0.35      1000
           5       0.66      0.48      0.56      1000

    accuracy                           0.43      5000
   macro avg       0.46      0.43      0.44      5000
weighted avg       0.46      0.43      0.44      5000



In [ ]:
print("Start running the model on Spanish")
test_dataloader, test_y=convert_test_data_of('es')
acc, mae, report=test_model_on(test_dataloader, test_y)
print(f'Test Accuracy: {acc:.3f}')
print(f'MAE: {mae:.3f}')
print(report)

Start running the model on Spanish

Testing...
Model state loaded from file saved_weights.pt
  Batch    50  of    313.
  Batch   100  of    313.
  Batch   150  of    313.
  Batch   200  of    313.
  Batch   250  of    313.
  Batch   300  of    313.
Test Accuracy: 44.760
MAE: 0.691
              precision    recall  f1-score   support

           1       0.70      0.45      0.55      1000
           2       0.38      0.30      0.33      1000
           3       0.34      0.51      0.41      1000
           4       0.40      0.56      0.46      1000
           5       0.66      0.41      0.51      1000

    accuracy                           0.45      5000
   macro avg       0.49      0.45      0.45      5000
weighted avg       0.49      0.45      0.45      5000



In [ ]:
print("Start running the model on Japanese")
test_dataloader, test_y=convert_test_data_of('ja')
acc, mae, report=test_model_on(test_dataloader, test_y)
print(f'Test Accuracy: {acc:.3f}')
print(f'MAE: {mae:.3f}')
print(report)

Start running the model on Japanese

Testing...
Model state loaded from file saved_weights.pt
  Batch    50  of    313.
  Batch   100  of    313.
  Batch   150  of    313.
  Batch   200  of    313.
  Batch   250  of    313.
  Batch   300  of    313.
Test Accuracy: 33.080
MAE: 1.171
              precision    recall  f1-score   support

           1       0.35      0.68      0.46      1000
           2       0.26      0.32      0.29      1000
           3       0.27      0.31      0.29      1000
           4       0.32      0.07      0.12      1000
           5       0.57      0.27      0.37      1000

    accuracy                           0.33      5000
   macro avg       0.36      0.33      0.31      5000
weighted avg       0.36      0.33      0.31      5000



In [ ]:
print("Start running the model on Chinese")
test_dataloader, test_y=convert_test_data_of('zh')
acc, mae, report=test_model_on(test_dataloader, test_y)
print(f'Test Accuracy: {acc:.3f}')
print(f'MAE: {mae:.3f}')
print(report)

Start running the model on Chinese

Testing...
Model state loaded from file saved_weights.pt
  Batch    50  of    313.
  Batch   100  of    313.
  Batch   150  of    313.
  Batch   200  of    313.
  Batch   250  of    313.
  Batch   300  of    313.
Test Accuracy: 34.700
MAE: 0.947
              precision    recall  f1-score   support

           1       0.50      0.52      0.51      1000
           2       0.30      0.40      0.34      1000
           3       0.27      0.46      0.34      1000
           4       0.28      0.19      0.23      1000
           5       0.65      0.17      0.27      1000

    accuracy                           0.35      5000
   macro avg       0.40      0.35      0.34      5000
weighted avg       0.40      0.35      0.34      5000

